In [66]:
#Start Application with Flask as the Web Framework of Choice
from flask import Flask, jsonify, request, render_template
from flask_cors import CORS
app = Flask(__name__)
# Add CORS to the API
CORS(app)

In [158]:
#2 Start Data Processing by obtaining excel file given as part of PWS
#Utilizing Padas
import pandas as pd
#Utilizing NumPy
import numpy as np
# Importing JSON for API 
import json

df = pd.DataFrame()
cols_to_use = [0,1,2,3,4,5,6,7]
csv_file_path = "../../assets/companylist.csv"
df = pd.read_csv(csv_file_path, 
    usecols= cols_to_use,
    encoding='utf-8'
    )
#IPOYear - initial public offering
#MarketCap - total value of all a company's shares of stock
#Sort Companies by Sector
df.sort_values(by=['Sector'], ascending=False).head()

#Identify how may unique sectors are actually in the data given
list_of_unique_sectors = df['Sector'].unique().tolist()
# print(list_of_unique_sectors)
# Count the number of Unique Sectors
count_of_unique_sectors = len(df['Sector'].unique().tolist())
# print(count_of_unique_sectors)
# Reviewed the List of Sectors to notice NaN for missing data. 
# Creating a new column for sector that will be clean as well as consolidating sectors to 10
# Migrating all Consumer Sectors to Consumer Industry and Technology with Energy.
df['Sector_Conv'] = np.where(df['Sector']=="Consumer Durables", 'Consumer Industry', df['Sector'])
df['Sector_Conv'] = np.where(df['Sector']=="Consumer Non-Durables", 'Consumer Industry', df['Sector'])
df['Sector_Conv'] = np.where(df['Sector']=='Consumer Services', 'Consumer Industry', df['Sector'])
df["Sector"] = df["Sector"].replace("Empty", "No Industry Identified")
df["Sector_Conv"] = df["Sector_Conv"].replace("Empty", "No Industry Identified")
df["Sector_Conv"] = df["Sector_Conv"].replace("Consumer Durables", "Consumer Industry")
df["Sector_Conv"] = df["Sector_Conv"].replace("Consumer Non-Durables", "Consumer Industry")
df["Sector_Conv"] = df["Sector_Conv"].replace("Technology", "Technology/Energy")
df["Sector_Conv"] = df["Sector_Conv"].replace("Energy", "Technology/Energy")

# Checking the conversion and consolidation
list_of_unique_sectors_conversion = df['Sector_Conv'].unique().tolist()
# print(list_of_unique_sectors_conversion)
# Checking the correct number of unique sectors
count_of_unique_sectors = len(df['Sector_Conv'].unique().tolist())
# print(count_of_unique_sectors)

# Remove NaN from to Sector Identified
df['Sector_Conv'] = df['Sector_Conv'].fillna('No Sector Identifed')

# Checking the conversion and consolidation - Count should be the same.
# list_of_unique_sectors_conversion = df['Sector_Conv'].unique().tolist()
# print(list_of_unique_sectors_conversion)

# Convert list to a JSON Object
# json_sector_list = json.dumps(list_of_unique_sectors_conversion)
df.head()

,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,Summary Quote,Sector_Conv
0,YI,"111, Inc.",8.5600,$699M,2018.0,Health Care,Medical/Nursing Services,https://www.nasdaq.com/symbol/yi,Health Care
1,PIH,"1347 Property Insurance Holdings, Inc.",5.7400,$34.51M,2014.0,Finance,Property-Casualty Insurers,https://www.nasdaq.com/symbol/pih,Finance
2,PIHPP,"1347 Property Insurance Holdings, Inc.",25.2499,$17.67M,NaN,Finance,Property-Casualty Insurers,https://www.nasdaq.com/symbol/pihpp,Finance
3,TURN,180 Degree Capital Corp.,1.9598,$60.99M,NaN,Finance,Finance/Investors Services,https://www.nasdaq.com/symbol/turn,Finance
4,FLWS,"1-800 FLOWERS.COM, Inc.",18.4600,$1.19B,1999.0,Consumer Services,Other Specialty Stores,https://www.nasdaq.com/symbol/flws,Consumer Industry


In [206]:
# Adding Sci Kit Learn Machine Learning Library
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
# Adding Visualization Library
from matplotlib import pyplot at plt
import seaborn as sns

# list_of_symbols = df['Symbol'].unique().tolist()
# list_of_symbols[:10]
# df.head()
# CART - Classification and Regression Trees. 
#Create the Classifier
# scikit learn uses real valued features
# features = [[3, Green], [3, Yellow], [1, Red], [1, Red], [3, Yellow]]
# features = [[140, 1], [130, 1], [150, 0], [170, 0]]
features = [[3, Green], [3, Yellow], [1, Red], [1, Red], [3, Yellow]]
# labels = [Orange, Orange, Apple, Apple]
# labels = [Apple, Apple, Grape, Grape, Lemon]
labels = [0, 0, 1, 1]
# Train the Classifier
clf = tree.DecisionTreeClassifier()
# we will setup a rule for a classifer where we tell fit to look at features and then labels.
clf = clf.fit(features, labels)
#fit - Find Patterns in data
print(clf.predict([[150, 0]]))

[1]


In [141]:
# Obtaining historical records from API for a company by utilizing their 
# symbols on the previous workbook.
import requests
import sklearn
from matplotlib import style
style.use("ggplot")
def get_stock_data_from_alphavantage_by_symbol(symbols):
    Alpha_Advantage_API = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=AMZN&outputsize=full&apikey=GJS1X1O8CLA8KA4H"
    resp = requests.get(Alpha_Advantage_API)
    if resp.status_code == 200:
#         print(resp.status_code)
        data = resp.json()
        data = data['Time Series (Daily)']
#         print(data)
        df = pd.DataFrame.from_dict(data, orient='columns')
        df = df.T
        return df.head()
    else:
        print("API did not pull any data from alphavantage")
# Executes the function
get_stock_data_from_alphavantage_by_symbol(list_of_symbols)
# Features - Also known as predicitors, input, or attributes; aka variables of data
# Features Matrix - Setup for more than one feature
# Response - aka The Target, label or Output. This is the output variable 
#            depending on the feature variable. 



,1. open,2. high,3. low,4. close,5. volume
2019-05-30,1825.4900,1829.4700,1807.8250,1816.3200,3076531
2019-05-29,1823.1200,1830.0000,1807.5300,1819.1900,4272379
2019-05-28,1832.7500,1849.2700,1827.3500,1836.4300,3199965
2019-05-24,1835.8900,1841.7600,1817.8500,1823.2800,3369673
2019-05-23,1836.5900,1844.0000,1804.2000,1815.4800,4424265


In [137]:

df['industry'] = df['industry'].fillna('No Industry Identified')
df['industry'].unique().tolist()
len(df['industry'].unique().tolist())

128

In [ ]:
# Count the number of unqiue companies
df['Name'] = df['Name'].fillna('No Company Identified')
df['Name'].unique().tolist()
# len(df['Name'].unique().tolist())

In [40]:
df.groupby(['Name','Sector','industry']).groups

{('111, Inc.',
  'Health Care',
  'Medical/Nursing Services'): Int64Index([0], dtype='int64'),
 ('1347 Property Insurance Holdings, Inc.',
  'Finance',
  'Property-Casualty Insurers'): Int64Index([1, 2], dtype='int64'),
 ('180 Degree Capital Corp.',
  'Finance',
  'Finance/Investors Services'): Int64Index([3], dtype='int64'),
 ('1-800 FLOWERS.COM, Inc.',
  'Consumer Services',
  'Other Specialty Stores'): Int64Index([4], dtype='int64'),
 ('1895 Bancorp of Wisconsin, Inc.',
  'Finance',
  'Banks'): Int64Index([5], dtype='int64'),
 ('1st Constitution Bancorp (NJ)',
  'Finance',
  'Savings Institutions'): Int64Index([6], dtype='int64'),
 ('1st Source Corporation',
  'Finance',
  'Major Banks'): Int64Index([7], dtype='int64'),
 ('21Vianet Group, Inc.',
  'Technology',
  'Computer Software: Programming, Data Processing'): Int64Index([8], dtype='int64'),
 ('2U, Inc.',
  'Technology',
  'Computer Software: Prepackaged Software'): Int64Index([9], dtype='int64'),
 ('360 Finance, Inc.',
  'Finan

In [140]:
# Getting list of 100 industries from 10 sectors. 
import requests
import pandas as pd
import numpy as np
import sys
import json
df = pd.DataFrame()

def get_avg_sector_rates():
    cols_to_use = [0,1,2,3,4,5,6,7]
    csv_file_path = "../../assets/companylist.csv"
    df = pd.read_csv(csv_file_path, 
        usecols= cols_to_use,
        encoding='utf-8'
        )
    df['IPOyear'] = df['IPOyear'].astype(str)
    df['IPOyear'] = df['IPOyear'].str.strip('.0')
    df = df.rename(index=str, columns = {"Symbol": "SYM", "Name": "CO.", "LastSale": "LP", "MarketCap": "MRKCAP", "IPOyear": "YR", "Sector": "SEC", "industry": "IND", "Summary Quote": "SUMQTE"})
    df['SYM'] = df['SYM'].str.strip()
    df['SEC'] = df['SEC'].str.strip()
    df['SEC_CONV'] = np.where(df['SEC']=="Consumer Durables", 'Consumer Industry', df['SEC'])
    df['SEC_CONV'] = np.where(df['SEC']=="Consumer Non-Durables", 'Consumer Industry', df['SEC'])
    df['SEC_CONV'] = np.where(df['SEC']=='Consumer Services', 'Consumer Industry', df['SEC'])
    df["SEC"] = df["SEC"].replace("Empty", "No Industry Identified")
    df["SEC_CONV"] = df["SEC_CONV"].replace("Empty", "No Industry Identified")
    df["SEC_CONV"] = df["SEC_CONV"].replace("Consumer Durables", "Consumer Industry")
    df["SEC_CONV"] = df["SEC_CONV"].replace("Consumer Non-Durables", "Consumer Industry")
    df["SEC_CONV"] = df["SEC_CONV"].replace("Technology", "Technology/Energy")
    df["SEC_CONV"] = df["SEC_CONV"].replace("Energy", "Technology/Energy")
    df = df.groupby('SEC_CONV').head(100)
#     df = df.iloc[:, [8,3]]
    df['MRKCAP'] = df['MRKCAP'].str.replace('$', '')
    df['MRKCAP'] = df['MRKCAP'].str.replace(',', '')
    df['MRKCAP'] = df['MRKCAP'].str.replace('M', ' M')
    df['MRKCAP'] = df['MRKCAP'].str.replace('B', ' B')
    df['SEC_CONV'] = df['SEC_CONV'].fillna(value='No Industry Identified')
    df['MRKCAP'] = df['MRKCAP'].fillna(value='0 D')
    df.dropna(inplace = True)
    df[['MRKCAP_AMT','MRKCAP_TYPE']] = df.MRKCAP.str.split(" ",expand=True,)
    millions = df['MRKCAP_AMT'].str.replace('.', '').fillna(value='0').astype(int).multiply(other = 100, fill_value = 5)
    billions = df['MRKCAP_AMT'].str.replace('.', '').fillna(value='0').astype(int).multiply(other = 1000000, fill_value = 5)
    df.loc[df['MRKCAP_TYPE'] == 'M', 'Conversion'] = millions
    df.loc[df['MRKCAP_TYPE'] == 'B', 'Conversion'] = billions
    df.loc[df['MRKCAP_TYPE'] == 'D', 'Conversion'] = 0
    df.loc[df['MRKCAP_TYPE'] == 'None', 'Conversion'] = df['MRKCAP_AMT'].fillna(value='0')
    df['Conversion'] = df['Conversion'].fillna(value='0')
    df['Conversion'] = df['Conversion'].astype(int)
    print(df.dtypes)
    print(sys.maxsize)
    group_sectors = df.groupby(['SEC_CONV','Conversion'])
    res = df.groupby(['SEC_CONV', 'CO.'])['Conversion'].mean().reset_index()
#     print(res)
    res = res.to_json(orient='columns')
    res = json.dumps(res)
    res = res.replace('\\','')
    res = res.replace('"{','{')
    res = res.replace('}"','}')
#   res.to_csv(r'c:/development/python\mrkcap.csv')
    return res
get_avg_sector_rates()

SYM             object
CO.             object
LP             float64
MRKCAP          object
YR              object
SEC             object
IND             object
SUMQTE          object
SEC_CONV        object
MRKCAP_AMT      object
MRKCAP_TYPE     object
Conversion       int32
dtype: object
9223372036854775807


'{"SEC_CONV":{"0":"Basic Industries","1":"Basic Industries","2":"Basic Industries","3":"Basic Industries","4":"Basic Industries","5":"Basic Industries","6":"Basic Industries","7":"Basic Industries","8":"Basic Industries","9":"Basic Industries","10":"Basic Industries","11":"Basic Industries","12":"Basic Industries","13":"Basic Industries","14":"Basic Industries","15":"Basic Industries","16":"Basic Industries","17":"Basic Industries","18":"Basic Industries","19":"Basic Industries","20":"Basic Industries","21":"Basic Industries","22":"Basic Industries","23":"Basic Industries","24":"Basic Industries","25":"Basic Industries","26":"Basic Industries","27":"Basic Industries","28":"Basic Industries","29":"Basic Industries","30":"Basic Industries","31":"Basic Industries","32":"Basic Industries","33":"Basic Industries","34":"Basic Industries","35":"Basic Industries","36":"Basic Industries","37":"Basic Industries","38":"Basic Industries","39":"Basic Industries","40":"Basic Industries","41":"Basic 

In [360]:
#API Data Pull
import pandas as pd
import requests
symbol = "YI"
def get_stock_data_from_alphavantage_by_symbol(symbol):
    # https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=MSFT&outputsize=full&apikey=
    Alpha_Advantage_API = "https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords=" + symbol + "&apikey=GJS1X1O8CLA8KA4H"
    resp = requests.get(Alpha_Advantage_API)
    if resp.status_code == 200:
        print(resp.status_code)
        data = resp.json()
#       print(data)
        data = data['bestMatches']
        df2 = pd.DataFrame.from_dict(data, orient='columns')
        df2.columns = ['SYM', 'CO.', 'Type', 'Country', 'LP-Time', 'LP-End-Time', 'UTC-Type', 'Dollar-Type', 'Last-List-Price']
        df2['SYM'] = df2['SYM'].str.strip()
        df2.loc[df2['SYM'] == str(symbol)]
        return df2
    else:
        print("API did not pull any data from alphavantage")
# Executes the function
get_stock_data_from_alphavantage_by_symbol(symbol)


200


,SYM,CO.,Type,Country,LP-Time,LP-End-Time,UTC-Type,Dollar-Type,Last-List-Price
0,YI,111 Inc.,Equity,United States,09:30,16:00,UTC-04,USD,1.0000
1,YINN,Direxion Daily FTSE China Bull 3X Shares,ETF,United States,09:30,16:00,UTC-04,USD,0.8000
2,YRD,Yirendai Ltd.,Equity,United States,09:30,16:00,UTC-04,USD,0.6667
3,YYY,YieldShares High Income ETF,ETF,United States,09:30,16:00,UTC-04,USD,0.6667
4,BOSQF,The Yield Growth Corp.,Equity,United States,09:30,16:00,UTC-04,USD,0.4000
5,YTEN,Yield10 Bioscience Inc.,Equity,United States,09:30,16:00,UTC-04,USD,0.4000
6,YGEHY,Yingli Green Energy Holding Company Limited,Equity,United States,09:30,16:00,UTC-04,USD,0.3333
7,YIN,Yintech Investment Holdings Limited,Equity,United States,09:30,16:00,UTC-04,USD,0.2857
8,YIPI,Yippy Inc.,Equity,United States,09:30,16:00,UTC-04,USD,0.1667


In [384]:
# Getting the 5 top companies of each sector with Market Cap
import requests
import pandas as pd
import numpy as np
import sys
import json
df = pd.DataFrame()

def get_top_companies_market_cap():
    cols_to_use = [0,1,2,3,4,5,6,7]
    csv_file_path = "../../assets/companylist.csv"
    df = pd.read_csv(csv_file_path, 
        usecols= cols_to_use,
        encoding='utf-8'
        )
    df['IPOyear'] = df['IPOyear'].astype(str)
    df['IPOyear'] = df['IPOyear'].str.strip('.0')
    df = df.rename(index=str, columns = {"Symbol": "SYM", "Name": "CO.", "LastSale": "LP", "MarketCap": "MRKCAP", "IPOyear": "YR", "Sector": "SEC", "industry": "IND", "Summary Quote": "SUMQTE"})
    df['SYM'] = df['SYM'].str.strip()
    df['SEC'] = df['SEC'].str.strip()
    df['SEC_CONV'] = np.where(df['SEC']=="Consumer Durables", 'Consumer Industry', df['SEC'])
    df['SEC_CONV'] = np.where(df['SEC']=="Consumer Non-Durables", 'Consumer Industry', df['SEC'])
    df['SEC_CONV'] = np.where(df['SEC']=='Consumer Services', 'Consumer Industry', df['SEC'])
    df["SEC"] = df["SEC"].replace("Empty", "No Industry Identified")
    df["SEC_CONV"] = df["SEC_CONV"].replace("Empty", "No Industry Identified")
    df["SEC_CONV"] = df["SEC_CONV"].replace("Consumer Durables", "Consumer Industry")
    df["SEC_CONV"] = df["SEC_CONV"].replace("Consumer Non-Durables", "Consumer Industry")
    df["SEC_CONV"] = df["SEC_CONV"].replace("Technology", "Technology/Energy")
    df["SEC_CONV"] = df["SEC_CONV"].replace("Energy", "Technology/Energy")
    df = df.groupby('SEC_CONV').head(100)
#     df = df.iloc[:, [8,3]]
    df['MRKCAP'] = df['MRKCAP'].str.replace('$', '')
    df['MRKCAP'] = df['MRKCAP'].str.replace(',', '')
    df['MRKCAP'] = df['MRKCAP'].str.replace('M', ' M')
    df['MRKCAP'] = df['MRKCAP'].str.replace('B', ' B')
    df['SEC_CONV'] = df['SEC_CONV'].fillna(value='No Industry Identified')
    df['MRKCAP'] = df['MRKCAP'].fillna(value='0 D')
    df.dropna(inplace = True)
    df[['MRKCAP_AMT','MRKCAP_TYPE']] = df.MRKCAP.str.split(" ",expand=True,)
    millions = df['MRKCAP_AMT'].str.replace('.', '').fillna(value='0').astype(int).multiply(other = 100, fill_value = 5)
    billions = df['MRKCAP_AMT'].str.replace('.', '').fillna(value='0').astype(int).multiply(other = 1000000, fill_value = 5)
    df.loc[df['MRKCAP_TYPE'] == 'M', 'Conversion'] = millions
    df.loc[df['MRKCAP_TYPE'] == 'B', 'Conversion'] = billions
    df.loc[df['MRKCAP_TYPE'] == 'D', 'Conversion'] = 0
    df.loc[df['MRKCAP_TYPE'] == 'None', 'Conversion'] = df['MRKCAP_AMT'].fillna(value='0')
    df['Conversion'] = df['Conversion'].fillna(value='0')
    df['Conversion'] = df['Conversion'].astype(int)
#     print(df.dtypes)
#     print(sys.maxsize)
#     group_sectors = df.groupby(['SEC_CONV','Conversion'])
#     res = df.groupby(['SEC_CONV'])['Conversion'].mean().reset_index()
    
#     print(res)
#     res = res.to_json(orient='columns')
#     res = json.dumps(res)
#     res = res.replace('\\','')
#     res = res.replace('"{','{')
#     res = res.replace('}"','}')
#   res.to_csv(r'c:/development/python\mrkcap.csv')
    return df
get_top_companies_market_cap()

,SYM,CO.,LP,MRKCAP,YR,SEC,IND,SUMQTE,SEC_CONV,MRKCAP_AMT,MRKCAP_TYPE,Conversion
0,YI,"111, Inc.",8.5600,699 M,2018,Health Care,Medical/Nursing Services,https://www.nasdaq.com/symbol/yi,Health Care,699,M,69900
1,PIH,"1347 Property Insurance Holdings, Inc.",5.7400,34.51 M,2014,Finance,Property-Casualty Insurers,https://www.nasdaq.com/symbol/pih,Finance,34.51,M,345100
2,PIHPP,"1347 Property Insurance Holdings, Inc.",25.2499,17.67 M,nan,Finance,Property-Casualty Insurers,https://www.nasdaq.com/symbol/pihpp,Finance,17.67,M,176700
3,TURN,180 Degree Capital Corp.,1.9598,60.99 M,nan,Finance,Finance/Investors Services,https://www.nasdaq.com/symbol/turn,Finance,60.99,M,609900
4,FLWS,"1-800 FLOWERS.COM, Inc.",18.4600,1.19 B,1999,Consumer Services,Other Specialty Stores,https://www.nasdaq.com/symbol/flws,Consumer Industry,1.19,B,119000000
5,BCOW,"1895 Bancorp of Wisconsin, Inc.",9.3727,45.71 M,2019,Finance,Banks,https://www.nasdaq.com/symbol/bcow,Finance,45.71,M,457100
6,FCCY,1st Constitution Bancorp (NJ),18.7500,161.77 M,nan,Finance,Savings Institutions,https://www.nasdaq.com/symbol/fccy,Finance,161.77,M,1617700
7,SRCE,1st Source Corporation,45.4700,1.17 B,nan,Finance,Major Banks,https://www.nasdaq.com/symbol/srce,Finance,1.17,B,117000000
8,VNET,"21Vianet Group, Inc.",7.5500,848.56 M,2011,Technology,"Computer Software: Programming, Data Processing",https://www.nasdaq.com/symbol/vnet,Technology/Energy,848.56,M,8485600
9,TWOU,"2U, Inc.",39.5200,2.49 B,2014,Technology,Computer Software: Prepackaged Software,https://www.nasdaq.com/symbol/twou,Technology/Energy,2.49,B,249000000


In [116]:
#Merging data that is found within the Second Dataframe from the API
def merge_company_data_for_single_api(sym1, sym2):
#     sym1 = get_stock_data_from_alphavantage_by_symbol()
#     sym2 = get_company_data_from_csv()
    return sym1.merge(sym2)

200


,SYM,CO.,Type,Country,LP-Time,LP-End-Time,UTC-Type,Dollar-Type,Last-List-Price,LP,MRKCAP,YR,SEC,IND,SUMQTE
0,YIN,Yintech Investment Holdings Limited,Equity,United States,09:30,16:00,UTC-04,USD,0.4000,4.95,$353.66M,2016,Finance,Investment Bankers/Brokers/Service,https://www.nasdaq.com/symbol/yin


In [178]:
#Info from API
import pandas as pd
import requests
def get_sector_stats():
    Alpha_Advantage_API_Sector = "https://www.alphavantage.co/query?function=SECTOR&apikey=GJS1X1O8CLA8KA4H"
    resp = requests.get(Alpha_Advantage_API_Sector)
    if resp.status_code == 200:
        print(resp.status_code)
        data = resp.json()
        df3 = pd.DataFrame.from_dict(data, orient='columns')
        df3.columns = ['SEC', 'Real_Time', '1_Day_Performance', '5_Day_Performance', '1_Month_Performance', '3_Month_Performance', 'YTD_Performance', '1_Year_Performance', '3_Year_Performance', '5_Year_Performance', '10_Year_Performance']
#         df3['SEC'] = df3['SEC'].str.strip()
#         df2.loc[df2['SYM'] == str(symbol)]
        df3 = df3['SEC']
        df3 = df3.replace('NaN', '')
        return df3
    else:
        print("API did not pull any data from alphavantage")
get_sector_stats()

200


Communication Services                                              NaN
Consumer Discretionary                                              NaN
Consumer Staples                                                    NaN
Energy                                                              NaN
Financials                                                          NaN
Health Care                                                         NaN
Industrials                                                         NaN
Information               US Sector Performance (realtime & historical)
Information Technology                                              NaN
Last Refreshed                                   04:20 PM ET 05/23/2019
Materials                                                           NaN
Real Estate                                                         NaN
Utilities                                                           NaN
Name: SEC, dtype: object

In [362]:
#Articles via API
import pandas as pd
import requests
def get_stock_news():
    stock_news_api = "https://stocknewsapi.com/api/v1?tickers=YI&items=50&type=article&token=qqygybi5zike7wfmu21lawdnfndqhhi53ndszk83"
    resp = requests.get(stock_news_api)
    if resp.status_code == 200:
        print(resp.status_code)
        data = resp.json()
        data = data['data']
        df4 = pd.DataFrame.from_dict(data, orient='columns')
        df4.columns = ['DATE', 'IMG_URL', 'NEWS_URL', 'Sentiment', 'Source_Name', 'Tags', 'Text', 'SYM', 'Title', 'Type']
        return df4
    else:
        print("API did not pull any data from alphavantage")
get_stock_news()

200


ValueError: Length mismatch: Expected axis has 0 elements, new values have 10 elements

In [131]:
#Web Scrap
import pandas as pd
import requests
from bs4 import BeautifulSoup
symbols = ['yi','bhf']
companyimages = []
nasdaq_website = "https://www.nasdaq.com/symbol/fgp"
page = requests.get(nasdaq_website)
soup = BeautifulSoup(page.text, 'html.parser')
review_text_elem = soup.find_all(class_='quote-symbol-icon')

for item in review_text_elem:
    companyimages.append(item)

companyimages
# review_text_elem



[]

In [63]:
#API Data Pull
import pandas as pd
import requests
from IPython.core import display as ICD
cols_to_use = [0,1,2,3,4,5,6,7]
csv_file_path = r"../../assets/companylist.csv"
df = pd.read_csv(csv_file_path, 
                 usecols= cols_to_use,
                 encoding='utf-8'
                )
df['IPOyear'] = df['IPOyear'].astype(str)
df['IPOyear'] = df['IPOyear'].str.strip('.0')
df = df.rename(index=str, columns = {"Symbol": "SYM", "Name": "CO.", "LastSale": "LP", "MarketCap": "MRKCAP", "IPOyear": "YR", "Sector": "SEC", "industry": "IND", "Summary Quote": "SUMQTE"})
arraySymbols = []
for x in range (0, len(df['SYM'])):
    arraySymbols.append(df.iloc[x]['SYM'])
    
#print (len(df['SYM']))
#print (arraySymbols)
#print (len(arraySymbols))
def combine_csv_api (symbol):
    Alpha_Advantage_API = "https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords=" + symbol + "&apikey=GJS1X1O8CLA8KA4H"
    resp = requests.get(Alpha_Advantage_API)
    
    if resp.status_code == 200:
        data = resp.json()
        print (data)
        data = data['bestMatches']
        
        df2 = pd.DataFrame.from_dict(data, orient='columns')
        df2.columns = ['SYM', 'CO.', 'Type', 'Country', 'LP-Time', 'LP-End-Time', 'UTC-Type', 'Dollar-Type', 'Last-List-Price']
        df2['SYM'] = df2['SYM'].str.strip()
        
        df3 = pd.merge(df.loc[df["SYM"] == str(symbol)], df2.loc[df2['SYM'] == str(symbol)], on="SYM")
        
        del df3['CO._y']
        
        df3.columns = ['SYM','CO.','LP','MRKCAP','YR','SEC','IND','SUMQTE', 'Type','Country','LP-Time', 'LP-End-Time', 'UTC-Type', 'Dollar-Type', 'Last-List-Price']
                
        return df3 #ICD.display(df3)
    
        #return (df.loc[df["SYM"] == "YI"])
    
    else:
        print("API did not pull any data from alphavantage")
# Executes the function    
for x in range (1, len(arraySymbols)):
    if x == 1:
        df4 = combine_csv_api (arraySymbols[x-1])
        df5 = combine_csv_api (arraySymbols[x])
        
        dfFinal = df4.append(df5)
    else:
        df6 = combine_csv_api (arraySymbols[x])
        dfFinal = dfFinal.append(df6)
print (dfFinal)

{'bestMatches': [{'1. symbol': 'YI', '2. name': '111 Inc.', '3. type': 'Equity', '4. region': 'United States', '5. marketOpen': '09:30', '6. marketClose': '16:00', '7. timezone': 'UTC-04', '8. currency': 'USD', '9. matchScore': '1.0000'}, {'1. symbol': 'YINN', '2. name': 'Direxion Daily FTSE China Bull 3X Shares', '3. type': 'ETF', '4. region': 'United States', '5. marketOpen': '09:30', '6. marketClose': '16:00', '7. timezone': 'UTC-04', '8. currency': 'USD', '9. matchScore': '0.8000'}, {'1. symbol': 'YRD', '2. name': 'Yirendai Ltd.', '3. type': 'Equity', '4. region': 'United States', '5. marketOpen': '09:30', '6. marketClose': '16:00', '7. timezone': 'UTC-04', '8. currency': 'USD', '9. matchScore': '0.6667'}, {'1. symbol': 'YYY', '2. name': 'YieldShares High Income ETF', '3. type': 'ETF', '4. region': 'United States', '5. marketOpen': '09:30', '6. marketClose': '16:00', '7. timezone': 'UTC-04', '8. currency': 'USD', '9. matchScore': '0.6667'}, {'1. symbol': 'BOSQF', '2. name': 'The Yi

KeyError: 'bestMatches'